## Импорты

In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.append('../evaluation/')

In [3]:
import os
import random
import warnings
import zipfile as zf
from copy import deepcopy
from pprint import pprint
from time import time
from typing import Any

import numpy as np
import pandas as pd
import requests
from IPython.display import display
from rectools import Columns
from rectools.dataset import Dataset, Interactions
from rectools.metrics import MAP, NDCG, MeanInvUserFreq, Precision, Recall, Serendipity, calc_metrics
from rectools.metrics.base import MetricAtK
from rectools.model_selection import Splitter, TimeRangeSplitter
from rectools.models import PopularModel, RandomModel
from rectools.models.base import ModelBase
from tqdm import tqdm

from metrics import calculate_metrics
from visualization import visualize_metrics

In [4]:
RANDOM_STATE = 42
random.seed(RANDOM_STATE)
os.environ["PYTHONHASHSEED"] = str(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [5]:
K_RECOS = 10
N_SPLITS = 3

## Инициализация датасета

In [6]:
url = "https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip"

In [7]:
req = requests.get(url, stream=True)

with open("kion.zip", "wb") as fd:
    total_size_in_bytes = int(req.headers.get("Content-Length", 0))
    progress_bar = tqdm(desc="kion dataset download", total=total_size_in_bytes, unit="iB", unit_scale=True)
    for chunk in req.iter_content(chunk_size=2**20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 78.6M/78.8M [00:45<00:00, 2.97MiB/s]

In [8]:
files = zf.ZipFile("kion.zip", "r")
files.extractall()
files.close()

Немного предобработаем датасет

In [9]:
interactions = pd.read_csv("data_original/interactions.csv", parse_dates=["last_watch_dt"])

interactions.rename(columns={"last_watch_dt": Columns.Datetime, "total_dur": Columns.Weight}, inplace=True)

In [10]:
interactions = Interactions(interactions)

In [11]:
interactions.df.head()

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250.0,72.0
1,699317,1659,2021-05-29,8317.0,100.0
2,656683,7107,2021-05-09,10.0,0.0
3,864613,7638,2021-07-05,14483.0,100.0
4,964868,9506,2021-04-30,6725.0,100.0


In [12]:
users = pd.read_csv("data_original/users.csv")
items = pd.read_csv("data_original/items.csv")

## Подсчет метрик

Инициализируем модели

In [13]:
models: dict[str, ModelBase] = {"random": RandomModel(random_state=RANDOM_STATE), "popular": PopularModel()}

Инициализиуем метрики

In [14]:
metrics: dict[str, MetricAtK] = {}
for k in [1, 5, 10]:
    metrics.update(
        {
            f"top@{k}_precision": Precision(k=k),
            f"top@{k}_recall": Recall(k=k),
            f"top@{k}_ndcg": NDCG(k=k),
            f"top@{k}_map": MAP(k=k),
            f"top@{k}_serendipity": Serendipity(k=k),
            f"top@{k}_mean_inv_user_freq": MeanInvUserFreq(k=k),
        }
    )

Инициализиурем splitter

In [15]:
splitter: Splitter = TimeRangeSplitter(
    test_size="7D",
    n_splits=N_SPLITS,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

Посчитаем метрики и визуализируем результаты обучения

In [ ]:
report = calculate_metrics(interactions=interactions, metrics=metrics, models=models, splitter=splitter, k_recos=K_RECOS)

kion dataset download: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78.8M/78.8M [01:00<00:00, 2.97MiB/s]

In [ ]:
visualize_metrics(report)

## Визуализация результатов обучения

2.1. Группировка по item_id (то что они жирным) особой ценности не несет (это индексы)
2.2. Имеет смысл сделать группировку по типу (реко или история). Может быть даже их цветами выделить чтобы легче читалось.
2.3. Ранки должны быть целыми
2.4. Стоит добавить в пример юзеров, у которых побольше айтемов в истории.
2.5. А также предусмотреть кейс, когда айтемов в истории слишком много - тогда надо отрезать, но оставлять не первые, а последние => сортировать историю надо не по возрастанию, а по убыванию datetime
2.6. Название столбца count неинформативное

In [27]:
def visualize(model: ModelBase, dataset: Dataset, user_ids: list[int], item_data: dict[str, str], k_recos: int):
    recos = model.recommend(users=user_ids, k=k_recos, dataset=dataset, filter_viewed=True)
    recos["type"] = "reco"
    recos.drop("score", axis=1, inplace=True)
    history = dataset.interactions.df[dataset.interactions.df["user_id"].isin(user_ids)].sort_values(
        ["user_id", "datetime"]
    )
    history["rank"] = (history.sort_values("datetime").groupby(["user_id"]).datetime.rank()).astype('int')
    history["type"] = "history"
    history.drop(["datetime", "weight"], axis=1, inplace=True)

    report = pd.concat([recos, history])
    count_views = dataset.interactions.df.groupby("item_id").count()["user_id"]
    report = report.merge(item_data, how="inner", on="item_id")
    count_views.name = "number_of_views"
    report = report.merge(count_views, how="inner", on="item_id")
    
    report.sort_values(["user_id", "type"], inplace=True)
    report.set_index(["user_id", "item_id"], inplace=True)
    return report

In [28]:
user_ids = [666262, 672861, 955527]
dataset = Dataset.construct(interactions.df)
item_data = items[["item_id", "title", "genres"]]

In [29]:
reports = {}
for model_name, model in models.items():
    init_model = deepcopy(model)
    init_model.fit(dataset)
    reports[model_name] = visualize(
        model=init_model, dataset=dataset, user_ids=user_ids, item_data=item_data, k_recos=K_RECOS
    )

In [30]:
for model_name, report in reports.items():
    pprint(f"Model name: {model_name}")
    display(report)

'Model name: random'


rank     type  \
user_id item_id                  
666262  93          1  history   
        7419        1     reco   
        9109        2     reco   
        13917       3     reco   
        13332       4     reco   
        1331        5     reco   
        15448       6     reco   
        1123        7     reco   
        9933        8     reco   
        3287        9     reco   
        8478       10     reco   
672861  25          1  history   
        32          2  history   
        1483        1     reco   
        11062       2     reco   
        12863       3     reco   
        15487       4     reco   
        1818        5     reco   
        14809       6     reco   
        8207        7     reco   
        8799        8     reco   
        14725       9     reco   
        11960      10     reco   
955527  21          1  history   
        11405       1     reco   
        14305       2     reco   
        7720        3     reco   
        1212        4     reco   
        8485        5     reco   
        12234       6     reco   
        12862       7     reco   
        6604        8     reco   
        5291        9     reco   
        6538       10     reco   

                                                             title  \
user_id item_id                                                      
666262  93                                    Дом ночных призраков   
        7419                                           Ода радости   
        9109                                       Последняя битва   
        13917                                          Преисподняя   
        13332                                            Лихорадка   
        1331                                              Вечность   
        15448                                                Леший   
        1123                                                Богема   
        9933                                                Шедевр   
        3287     Единоборства для детей (3-6 лет) Школа героев ...   
        8478                       Американский пирог: Все в сборе   
672861  25                           Медвежонок Винни и его друзья   
        32                                          В ритме сердца   
        1483                                     Джек: Путь смерти   
        11062                                               Фиаско   
        12863                               Песня Любви (субтитры)   
        15487                                               Палата   
        1818                     Перспектива (с тифлокомментарием)   
        14809                                             Красотка   
        8207                   Смешарики.Легенда о золотом драконе   
        8799                                 Тихоокеанский рубеж 2   
        14725                                        Расти-механик   
        11960                           Учим математику с Ам Нямом   
955527  21                                             Признание 5   
        11405    Леопард, великолепная способность к адаптации....   
        14305                                    Творение Господне   
        7720                                     Мусульмане как мы   
        1212                                 Проклятие ведьмы снов   
        8485                              Искусство путешествовать   
        12234                                              Мозаика   
        12862                                        Мороз по коже   
        6604                                       Класс коррекции   
        5291                                      Договор молчания   
        6538                                    Третья волна зомби   

                                                            genres  \
user_id item_id                                                      
666262  93                  зарубежные, криминал, детективы, ужасы   
        7419                                               ко

'Model name: popular'


rank     type                          title  \
user_id item_id                                                 
666262  93          1  history           Дом ночных призраков   
        10440       1     reco                    Хрустальный   
        15297       2     reco                Клиника счастья   
        9728        3     reco              Гнев человеческий   
        13865       4     reco                       Девятаев   
        4151        5     reco         Секреты семейной жизни   
        3734        6     reco   Прабабушка легкого поведения   
        2657        7     reco                     Подслушано   
        4880        8     reco                          Афера   
        142         9     reco                           Маша   
        6809       10     reco                          Дуров   
672861  25          1  history  Медвежонок Винни и его друзья   
        32          2  history                 В ритме сердца   
        10440       1     reco                    Хрустальный   
        15297       2     reco                Клиника счастья   
        9728        3     reco              Гнев человеческий   
        13865       4     reco                       Девятаев   
        4151        5     reco         Секреты семейной жизни   
        3734        6     reco   Прабабушка легкого поведения   
        2657        7     reco                     Подслушано   
        4880        8     reco                          Афера   
        142         9     reco                           Маша   
        6809       10     reco                          Дуров   
955527  21          1  history                    Признание 5   
        10440       1     reco                    Хрустальный   
        15297       2     reco                Клиника счастья   
        9728        3     reco              Гнев человеческий   
        13865       4     reco                       Девятаев   
        4151        5     reco         Секреты семейной жизни   
        3734        6     reco   Прабабушка легкого поведения   
        2657        7     reco                     Подслушано   
        4880        8     reco                          Афера   
        142         9     reco                           Маша   
        6809       10     reco                          Дуров   

                                                    genres  number_of_views  
user_id item_id                                                              
666262  93          зарубежные, криминал, детективы, ужасы            68581  
        10440                          триллеры, детективы                7  
        15297                             драмы, мелодрамы                1  
        9728                             боевики, триллеры                7  
        13865                  драмы, военные, приключения                3  
        4151                                       комедии              325  
        3734                                       комедии               23  
        2657                               драмы, триллеры              285  
        4880                                       комедии               39  
        142                                драмы, триллеры            55043  
        6809                                документальное               96  
672861  25       мюзиклы, мультфильм, приключения, комедии           202457  
        32                       драмы, мюзиклы, мелодрамы           132865  
        10440                          триллеры, детективы                7  
        15297                             драмы, мелодрамы                1  
        9728                             боевики, триллеры                7  
        13865                  драмы, военные, приключения                3  
        4151                                       комедии              325  
        3734                                       комедии               23  
        2657                          